In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
building_data = pd.read_parquet("//Users/adrian/Documents/ICAI/TFG/Space-Conditioning-Electrification/data/consumption/MA_baseline_metadata_and_annual_results.parquet")

# Heating

In [4]:
col_list = [
    "in.heating_setpoint",
    "in.heating_setpoint_has_offset",
    "in.heating_setpoint_offset_magnitude",
    "in.heating_setpoint_offset_period",
]

heating_data = building_data[col_list]

In [6]:
def generate_building_schedule(mode: str, building_id: int, building_data: pd.DataFrame) -> np.ndarray:
    """Generates a building's heating setpoint offset schedule

    Args:
        mode (str): Either heating or cooling
        building_id (int): ID of the building
        building_data (pd.DataFrame): Heating setpoint dataset

    Returns:
        np.ndarray: Building's heating schedule during the day (when to apply offset)
    """
    building_data = building_data[building_data.index == building_id]

    night_start = 22
    night_end = 7

    day_start = 9
    day_end = 17

    schedule = np.zeros(24)

    has_offset = "in.heating_setpoint_has_offset" if mode == "heating" else "in.cooling_setpoint_has_offset"
    offset_magnitude = "in.heating_setpoint_offset_magnitude" if mode == "heating" else "in.cooling_setpoint_offset_magnitude"
    offset_period = "in.heating_setpoint_offset_period" if mode == "heating" else "in.cooling_setpoint_offset_period"

    if building_data[has_offset].item() == "Yes":
        offset_magnitude = int(building_data[offset_magnitude].item().replace("F", ""))
        offset_period = building_data[offset_period].item()

        if offset_period[-1] == "h":
            offset_period_number = int(offset_period[-3:-1])
        else:
            offset_period_number = 0

        

        if "Night" in offset_period:
            night_start += offset_period_number
            night_end += offset_period_number

            if night_start > night_end:
                schedule[night_start:] = 1
                schedule[:night_end] = 1
            else:
                schedule[night_start:night_end] = 1

        if "Day" in offset_period:
            day_start += offset_period_number
            day_end += offset_period_number

            if day_start > day_end:
                schedule[day_start:] = 1
                schedule[:day_end] = 1
            else:
                schedule[day_start:day_end] = 1

        schedule *= offset_magnitude

    return schedule


In [7]:
def generate_building_setpoint_timeseries(mode: str, building_id: int, building_data: pd.DataFrame) -> pd.DataFrame:
    """Generates a building's heating setpoint timeseries

    Args:
        mode (str): Either heating or cooling
        building_id (int): ID of the building
        building_data (pd.DataFrame): Heating setpoint dataset

    Returns:
        pd.DataFrame: Building's heating setpoint timeseries
    """
    building_data = building_data[building_data.index == building_id]

    setpoint = "in.heating_setpoint" if mode == "heating" else "in.cooling_setpoint"

    # Generate empty timeseries for a full year (2018)
    timeseries = pd.date_range(start="2018-01-01", end="2018-12-31", freq="h")
    timeseries = pd.DataFrame(timeseries, columns=["timestamp"])
    timeseries["setpoint"] = int(building_data[setpoint].item().replace("F", ""))

    schedule = generate_building_schedule(mode, building_id, building_data)

    def apply_offset(row, schedule):
        hour = row["timestamp"].hour
        return row["setpoint"] + schedule[hour]
    
    timeseries["setpoint"] = timeseries.apply(apply_offset, axis=1, schedule=schedule)

    return timeseries

    

In [12]:
full_heating_timeseries_dataset = [None for _ in range(len(heating_data))]
for i in range(len(heating_data)):
    print(f"Processing building {i}/{len(heating_data)}           ", end="\r")
    full_heating_timeseries_dataset[i] = generate_building_setpoint_timeseries("heating", heating_data.index[i], heating_data)

# Cooling

In [9]:
col_list = [
    "in.cooling_setpoint",
    "in.cooling_setpoint_has_offset",
    "in.cooling_setpoint_offset_magnitude",
    "in.cooling_setpoint_offset_period",
]

cooling_data = building_data[col_list]

In [10]:
full_cooling_timeseries_dataset = [None for _ in range(len(cooling_data))]
for i in range(len(cooling_data)):
    print(f"Processing building {i}/{len(cooling_data)}           ", end="\r")
    full_cooling_timeseries_dataset[i] = generate_building_setpoint_timeseries("cooling", cooling_data.index[i], cooling_data)

# Combine timeseries

In [13]:
full_timeseries = full_heating_timeseries_dataset

for i in range(len(full_cooling_timeseries_dataset)):
    # Change values between June 1st and October 31st to cooling setpoints
    full_timeseries[i].loc["2018-06-01":"2018-10-31", "setpoint"] = full_cooling_timeseries_dataset[i].loc["2018-06-01":"2018-10-31", "setpoint"]

In [20]:
timeseries_1 = full_timeseries[0]
heating_1 = full_heating_timeseries_dataset[0]
cooling_1 = full_cooling_timeseries_dataset[0]

print(heating_1["setpoint"].value_counts())

setpoint
67.0    8737
Name: count, dtype: int64
